<h1 align="center"><font color="red">Evaluating RAG Applications with RAGAs</font></h1>

### <font color="yellow">Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Link de estudo:

* [Nov 14, 2023: Retrieval-Augmented Generation (RAG): From Theory to LangChain Implementation](https://towardsdatascience.com/retrieval-augmented-generation-rag-from-theory-to-langchain-implementation-4e9bd5f6a4f2)

* [Dec 13, 2023: Evaluating RAG Applications with RAGAs](https://towardsdatascience.com/evaluating-rag-applications-with-ragas-81d67b0ee31a)

* [Evaluaing LangChain QA Chains](https://docs.ragas.io/en/latest/howtos/integrations/langchain.html#)

# <font color="pink">O que é `RAGAs`</font>

<font color="orange">`RAGAs` (`R`etrieval - `A`ugmented `G`eneration `As`sessment) é um Framework ([GitHub](https://github.com/explodinggradients/ragas), [Docs](https://docs.ragas.io/en/latest/)) que fornece os ingredientes necessários para ajudá-lo a avaliar seu `pipeline RAG` em um nível de componente .</font>

In [ ]:
%pip install -qU openai langchain==0.1.11 tiktoken qdrant-client # transformers sentence-transformers
%pip install ragas==0.0.22 -qq

## <font color="gree">Geração Aumentada de Recuperação (RAG): Da Teoria à Implementação LangChain</font>

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file


In [2]:
import requests
from langchain.document_loaders import TextLoader

# url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"
# res = requests.get(url)
# with open("state_of_the_union.txt", "w") as f:
#     f.write(res.text)

loader = TextLoader('./data/1-Administracao_digital.txt')
documents = loader.load()

In [3]:
documents

[Document(page_content='ACUMULADOR DE ENTULHO \nO que é o serviço? A acumulação de lixo/entulho é um problema de saúde pública. Você pode solicitar à Secretaria de Saúde do DF uma visita à área de acumuladores para analisar o caso e iniciar as ações para a solução do problema. \nInformações importantes do serviço: As visitas técnicas/inspeções podem ser solicitadas em qualquer uma das seguintes situações: \n• Inspeção ambiental, residencial e comercial, e em órgãos públicos para controle de vetores, peçonhentos, sinantrópicos para controle pragas urbanas de importância médica; \n• Pesquisas vetoriais técnicas aos imóveis do distrito federal, para identificar e avaliar as condições ambientais que venham a representar riscos à saúde humana; \n• Para avaliação de roedores, animais peçonhentos, sinantrópicos nos imóveis residenciais, comerciais e órgãos públicos e possíveis providencias; \n• Para avaliação e recomendações sobre pombos na residência; \n• Para avaliação sobre roedores na res

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=600,
                                      chunk_overlap=60
                                     )

chunks = text_splitter.split_documents(documents)


In [ ]:
chunks

In [6]:
len(chunks)

33

In [7]:
#%pip install weaviate-client

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions


client = weaviate.Client(embedded_options = EmbeddedOptions())


vectorstore = Weaviate.from_documents(client = client,    
                                      documents = chunks,
                                      embedding = OpenAIEmbeddings(),
                                      by_text = False
                                     )


In [10]:
retriever = vectorstore.as_retriever()

In [11]:
from langchain.prompts import ChatPromptTemplate

template = """Você é um assistente para tarefas de resposta a perguntas.
Use as seguintes partes do contexto recuperado para responder à pergunta.
Se você não sabe a resposta, basta dizer que não sabe.
Use no máximo duas frases e mantenha a resposta concisa.
Question: {question} 
Context: {context} 
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)


In [12]:
print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Você é um assistente para tarefas de resposta a perguntas.\nUse as seguintes partes do contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, basta dizer que não sabe.\nUse no máximo duas frases e mantenha a resposta concisa.\nQuestion: {question} \nContext: {context} \nAnswer:\n'))]


In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)



In [18]:
query = "A quem beneficia a limpeza de Entulho?"

rag_chain.invoke(query)

'A limpeza de entulho beneficia a saúde pública e contribui para a conservação do ambiente urbano.'

In [19]:
query = "Quem foi Pelé?"

rag_chain.invoke(query)

'Não sei.'

## <font color="gree">Evaluating RAG Applications with RAGAs</font>

In [20]:
import requests
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"
# res = requests.get(url)
# with open("state_of_the_union.txt", "w") as f:
#     f.write(res.text)


# Load the data
loader = TextLoader('./data/1-Administracao_digital.txt')
documents = loader.load()

# Chunk the data
text_splitter = CharacterTextSplitter(chunk_size=600, chunk_overlap=60)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1856, which is longer than the specified 600
Created a chunk of size 2023, which is longer than the specified 600
Created a chunk of size 1134, which is longer than the specified 600
Created a chunk of size 1235, which is longer than the specified 600
Created a chunk of size 1513, which is longer than the specified 600
Created a chunk of size 1345, which is longer than the specified 600
Created a chunk of size 1252, which is longer than the specified 600
Created a chunk of size 844, which is longer than the specified 600
Created a chunk of size 2153, which is longer than the specified 600
Created a chunk of size 1294, which is longer than the specified 600
Created a chunk of size 1064, which is longer than the specified 600
Created a chunk of size 1151, which is longer than the specified 600
Created a chunk of size 1329, which is longer than the specified 600
Created a chunk of size 1202, which is longer than the specified 600
Created a chunk of size 1018, which

In [21]:
chunks

[Document(page_content='ACUMULADOR DE ENTULHO \nO que é o serviço? A acumulação de lixo/entulho é um problema de saúde pública. Você pode solicitar à Secretaria de Saúde do DF uma visita à área de acumuladores para analisar o caso e iniciar as ações para a solução do problema. \nInformações importantes do serviço: As visitas técnicas/inspeções podem ser solicitadas em qualquer uma das seguintes situações: \n• Inspeção ambiental, residencial e comercial, e em órgãos públicos para controle de vetores, peçonhentos, sinantrópicos para controle pragas urbanas de importância médica; \n• Pesquisas vetoriais técnicas aos imóveis do distrito federal, para identificar e avaliar as condições ambientais que venham a representar riscos à saúde humana; \n• Para avaliação de roedores, animais peçonhentos, sinantrópicos nos imóveis residenciais, comerciais e órgãos públicos e possíveis providencias; \n• Para avaliação e recomendações sobre pombos na residência; \n• Para avaliação sobre roedores na res

In [22]:
len(chunks)

33

In [23]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions
from dotenv import load_dotenv,find_dotenv


# Setup vector database
client = weaviate.Client(embedded_options = EmbeddedOptions())


# Populate vector database
vectorstore = Weaviate.from_documents(client = client,
                                      documents = chunks,
                                      embedding = OpenAIEmbeddings(),
                                      by_text = False
                                     )


# Define vectorstore as retriever to enable semantic search
retriever = vectorstore.as_retriever()

embedded weaviate is already listening on port 8079


{"level":"info","msg":"Created shard langchain_74059867733b4b758123d89d04124258_BhxvrKmaBoXq in 446.025µs","time":"2024-04-07T01:01:22-03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-04-07T01:01:22-03:00","took":51260}


In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# Define LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo",
                 temperature=0
                )


# Define prompt template
template = """Você é um assistente para tarefas de resposta a perguntas.
Use as seguintes partes do contexto recuperado para responder à pergunta.
Se você não sabe a resposta, basta dizer que não sabe.
Use no máximo duas frases e mantenha a resposta concisa.
Question: {question} 
Context: {context} 
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

{"action":"restapi_management","level":"info","msg":"Shutting down... ","time":"2024-04-07T01:01:28-03:00"}
{"action":"restapi_management","level":"info","msg":"Stopped serving weaviate at http://127.0.0.1:8079","time":"2024-04-07T01:01:28-03:00"}


### <font color="red">Preparando os dados de avaliação - RAGAs</font>

<font color="orange">Como os `RAGAs` pretendem ser um Framework de avaliação sem referências, os preparativos necessários do conjunto de dados de avaliação são mínimos. Você precisará preparar pares `question` e `ground_truths` a partir dos quais poderá preparar as informações restantes por meio de inferência da seguinte forma:</font>

In [38]:
from datasets import Dataset

questions = [
             "O que é Acumulador de entulho?",
             "O que é o serviço de limpeza de Grelha?",
             "Como solicitar o serviço de implantação, manutenção e/ou reforma de ciclovias do DF?",
             "Prazo pra prestação do serviço de criação, revitalização, remoção ou mudança Faixa de pedestre / Sinalização Viária?",
             "Prazo pra prestação do serviço de Iluminação pública?",
             "O que é Instalação de Parquinhos infantis?",
            ]

ground_truths = [
                ["Acumulador de entulho é um problema de saúde pública que pode ser solicitado à Secretaria de Saúde do DF para análise e ações de solução do problema."],
                ["O serviço de limpeza de grelha é a remoção de resíduos e sujeira das grelhas localizadas em vias públicas, garantindo o bom funcionamento do sistema de drenagem e evitando alagamentos."], 
                ["Você pode solicitar por meio do Portal de Serviços do GDF (https://servicos.df.gov.br/categorias?categoria=ADMINISTRA%C3%87%C3%83O%20REGIONAL%2024h&persona=Cidad%C3%A3o#servicos), através do telefone 156 ou pelo aplicativo eGDF, que você está utilizando. Para solicitar, clique no ícone de ""casinha"", na barra inferior, em seguida, clique no botão ""informe aqui"", lá você terá acesso a todos os serviços de forma simples e prática."],
                ["O prazo é superior à 30 (trinta) dias podendo variar de acordo com a demanda. Ver detalhamento dos prazos na carta de serviço do órgão."],
                ["Os prazos para as ordens de serviço variam de acordo com a demanda."],
                ["É uma garrafa de cerveja"]  #"É a instalação e manutenção dos equipamentos referentes aos parquinhos infantis em área pública.", # É uma garrafa de cerveja.
               ]

answers = []
contexts = []

# Inference
for query in questions:
  answers.append(rag_chain.invoke(query))
  contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}


data


{'question': ['O que é Acumulador de entulho?',
  'O que é o serviço de limpeza de Grelha?',
  'Como solicitar o serviço de implantação, manutenção e/ou reforma de ciclovias do DF?',
  'Prazo pra prestação do serviço de criação, revitalização, remoção ou mudança Faixa de pedestre / Sinalização Viária?',
  'Prazo pra prestação do serviço de Iluminação pública?',
  'O que é Instalação de Parquinhos infantis?'],
 'answer': ['O Acumulador de entulho é um serviço para lidar com a acumulação de lixo/entulho, considerado um problema de saúde pública. É possível solicitar uma visita à área de acumuladores para análise e ações de solução do problema.',
  'O serviço de limpeza de grelha consiste na reposição e limpeza de tampas de boca de lobo, bueiros e grelhas de ferro ou concreto em vias públicas.',
  'Você pode solicitar o serviço de implantação, manutenção e/ou reforma de ciclovias do DF através do Portal de Serviços do GDF, telefone 156 ou pelo aplicativo eGDF.',
  'O prazo para prestação 

In [39]:
# Convert dict to dataset
dataset = Dataset.from_dict(data)

dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 6
})

In [40]:
# Aqui vou passar para DataFrame, só para ter uma maior claridade:
import pandas as pd

df_eddy = pd.DataFrame(dataset)
df_eddy.head(10)


,question,answer,contexts,ground_truths
0,O que é Acumulador de entulho?,O Acumulador de entulho é um serviço para lida...,[ACUMULADOR DE ENTULHO \nO que é o serviço? A ...,[Acumulador de entulho é um problema de saúde ...
1,O que é o serviço de limpeza de Grelha?,O serviço de limpeza de grelha consiste na rep...,[BUEIRO/BOCA DE LOBO/GRELHA/MEIO-FIO/TAMPA DE ...,[O serviço de limpeza de grelha é a remoção de...
2,"Como solicitar o serviço de implantação, manut...","Você pode solicitar o serviço de implantação, ...",[CICLOVIA OU CICLOFAIXA EM VIA URBANA \nO que ...,[Você pode solicitar por meio do Portal de Ser...
3,"Prazo pra prestação do serviço de criação, rev...","O prazo para prestação do serviço de criação, ...",[FAIXA DE PEDESTRE / SINALIZAÇÃO VIÁRIA \nO qu...,[O prazo é superior à 30 (trinta) dias podendo...
4,Prazo pra prestação do serviço de Iluminação p...,O prazo para prestação do serviço de Iluminaçã...,[ILUMINAÇÃO PÚBLICA \nO que é o serviço? Compr...,[Os prazos para as ordens de serviço variam de...
5,O que é Instalação de Parquinhos infantis?,Instalação de Parquinhos infantis é a instalaç...,[PARQUINHOS INFANTIS \nO que é o serviço? É a ...,[É uma garrafa de cerveja]


In [41]:
df_eddy.shape


(6, 4)

### <font color="red">Avaliando o aplicativo RAG</font>

<font color="orange">Primeiro, importe todas as MÉTRICAS que deseja usar do `ragas.metrics`. Então, você pode usar a função `evaluate()` e simplesmente passar as métricas relevantes e o conjunto de dados preparado.</font>

In [42]:
from ragas import evaluate

from ragas.metrics import (faithfulness,
                           answer_relevancy,
                           context_recall,
                           context_precision,
                          )


result = evaluate(dataset = dataset, 
                  metrics=[context_precision,
                           context_recall,
                           faithfulness,
                           answer_relevancy,
                          ],
                 )


result

evaluating with [context_precision]


100%|██████████| 1/1 [00:01<00:00,  1.90s/it]


evaluating with [context_recall]


100%|██████████| 1/1 [00:03<00:00,  3.30s/it]


evaluating with [faithfulness]


100%|██████████| 1/1 [00:05<00:00,  5.01s/it]


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:06<00:00,  6.11s/it]


{'context_precision': 0.1667, 'context_recall': 0.8333, 'faithfulness': 0.9444, 'answer_relevancy': 0.9077}

In [43]:
df = result.to_pandas()

df.head(10)

,question,answer,contexts,ground_truths,context_precision,context_recall,faithfulness,answer_relevancy
0,O que é Acumulador de entulho?,O Acumulador de entulho é um serviço para lida...,[ACUMULADOR DE ENTULHO \nO que é o serviço? A ...,[Acumulador de entulho é um problema de saúde ...,0.0,1.0,1.000000,0.818544
1,O que é o serviço de limpeza de Grelha?,O serviço de limpeza de grelha consiste na rep...,[BUEIRO/BOCA DE LOBO/GRELHA/MEIO-FIO/TAMPA DE ...,[O serviço de limpeza de grelha é a remoção de...,0.0,1.0,0.666667,0.963392
2,"Como solicitar o serviço de implantação, manut...","Você pode solicitar o serviço de implantação, ...",[CICLOVIA OU CICLOFAIXA EM VIA URBANA \nO que ...,[Você pode solicitar por meio do Portal de Ser...,1.0,1.0,1.000000,0.893880
3,"Prazo pra prestação do serviço de criação, rev...","O prazo para prestação do serviço de criação, ...",[FAIXA DE PEDESTRE / SINALIZAÇÃO VIÁRIA \nO qu...,[O prazo é superior à 30 (trinta) dias podendo...,0.0,1.0,1.000000,0.896015
4,Prazo pra prestação do serviço de Iluminação p...,O prazo para prestação do serviço de Iluminaçã...,[ILUMINAÇÃO PÚBLICA \nO que é o serviço? Compr...,[Os prazos para as ordens de serviço variam de...,0.0,1.0,1.000000,0.905807
5,O que é Instalação de Parquinhos infantis?,Instalação de Parquinhos infantis é a instalaç...,[PARQUINHOS INFANTIS \nO que é o serviço? É a ...,[É uma garrafa de cerveja],0.0,0.0,1.000000,0.968439


In [37]:
df = result.to_pandas()

df.head(10)

,question,answer,contexts,ground_truths,context_precision,context_recall,faithfulness,answer_relevancy
0,O que é Acumulador de entulho?,O Acumulador de entulho é um problema de saúde...,[ACUMULADOR DE ENTULHO \nO que é o serviço? A ...,[Acumulador de entulho é um problema de saúde ...,0.0,1.0,1.000000,0.756356
1,O que é o serviço de limpeza de Grelha?,O serviço de limpeza de grelha consiste na rep...,[BUEIRO/BOCA DE LOBO/GRELHA/MEIO-FIO/TAMPA DE ...,[O serviço de limpeza de grelha é a remoção de...,0.0,1.0,0.666667,0.963392
2,"Como solicitar o serviço de implantação, manut...","Você pode solicitar o serviço de implantação, ...",[CICLOVIA OU CICLOFAIXA EM VIA URBANA \nO que ...,[Você pode solicitar por meio do Portal de Ser...,1.0,1.0,1.000000,0.893865
3,"Prazo pra prestação do serviço de criação, rev...","O prazo para prestação do serviço de criação, ...",[FAIXA DE PEDESTRE / SINALIZAÇÃO VIÁRIA \nO qu...,[O prazo é superior à 30 (trinta) dias podendo...,0.0,1.0,1.000000,0.896193
4,Prazo pra prestação do serviço de Iluminação p...,O prazo para prestação do serviço de Iluminaçã...,[ILUMINAÇÃO PÚBLICA \nO que é o serviço? Compr...,[Os prazos para as ordens de serviço variam de...,0.0,1.0,1.000000,0.905807
5,O que é Instalação de Parquinhos infantis?,Instalação de Parquinhos infantis é a instalaç...,[PARQUINHOS INFANTIS \nO que é o serviço? É a ...,[É a instalação e manutenção dos equipamentos ...,0.0,1.0,1.000000,0.968451
